In [ ]:
!pip install ultralytics roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 129.1 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="iikqpaDOob8gBYWthRdt")
project = rf.workspace("khalil-akremi-syexl").project("well-detection-101")
version = project.version(1)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to well-detection-101-1 in yolov8:: 100%|██████████| 320/320 [00:00<00:00, 12057.90it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Copy your dataset from Drive to local Colab for speed
!cp -r "/content/drive/MyDrive/data_rabies_101" /content/rabies_local

In [ ]:
from ultralytics import YOLO

# 1. Load the pre-trained weights
model = YOLO('yolov8n.pt')

# 2. Train on your 155 images using your Roboflow dataset
# Note: Ensure 'dataset' is defined from your Roboflow download cell
model.train(data=f"{dataset.location}/data.yaml", epochs=50, imgsz=640)

Ultralytics 8.3.240 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/well-detection-101-1/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=T

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fee6423a5a0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [ ]:
import cv2
import os
from pathlib import Path
from ultralytics import YOLO

# --- SETTINGS ---
# 1. Find the best weights from your YOLO training
runs_path = Path("runs/detect")
latest_train = sorted(list(runs_path.glob("train*")))[-1]
best_weights = latest_train / "weights" / "best.pt"

# 2. Define where the raw images are (Roboflow usually puts them here)
# If your images are in your Drive, change this to your Drive path
raw_data_path = Path(dataset.location)

# 3. Define the output path on Drive
output_base = Path("/content/drive/MyDrive/annotaion_101")
output_base.mkdir(parents=True, exist_ok=True)

# Load the model
well_model = YOLO(str(best_weights))

# Define common image extensions
valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff')

print(f"Scanning for images in: {raw_data_path}")

# Loop through every file in the dataset directory recursively
count = 0
for img_path in raw_data_path.rglob("*"):
    if img_path.suffix.lower() in valid_extensions:
        img = cv2.imread(str(img_path))
        if img is None: continue # Skip files that can't be read

        # Run YOLO detection
        results = well_model(img_path)[0]

        if len(results.boxes) > 0:
            # Get the coordinates for the 'Well' (top confidence detection)
            box = results.boxes.xyxy[0].cpu().numpy()
            x1, y1, x2, y2 = map(int, box)

            # Crop the image to the Well boundary
            crop = img[y1:y2, x1:x2]

            # --- STRUCTURE LOGIC ---
            # This part keeps your Positive/Negative folders organized
            # It looks for 'positive' or 'negative' in the folder path
            category = "uncategorized"
            if "positive" in str(img_path).lower():
                category = "positive"
            elif "negative" in str(img_path).lower():
                category = "negative"

            save_dir = output_base / category
            save_dir.mkdir(parents=True, exist_ok=True)

            # Save the cropped image
            save_name = f"cropped_{img_path.stem}.png" # Saving as .png preserves quality
            cv2.imwrite(str(save_dir / save_name), crop)
            count += 1

print(f"Done! Successfully cropped {count} images and saved them to {output_base}")

Scanning for images in: /content/well-detection-101-1

image 1/1 /content/well-detection-101-1/train/images/20250228_143815_jpg.rf.a7a4d5375515602369dbec2ae6997452.jpg: 640x640 1 well, 23.6ms
Speed: 17.7ms preprocess, 23.6ms inference, 6.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/well-detection-101-1/train/images/20250220_143816_jpg.rf.8a06f920ba234f311dfb3fc4aad9632f.jpg: 640x640 1 well, 20.8ms
Speed: 12.9ms preprocess, 20.8ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/well-detection-101-1/train/images/IMG_20250515_104058_jpg.rf.b941c6795141aeb2dab74f36b2944829.jpg: 640x640 1 well, 15.7ms
Speed: 8.0ms preprocess, 15.7ms inference, 3.8ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/well-detection-101-1/train/images/1000001394_jpg.rf.c5bdac53f8789206e2a88e2bc1682338.jpg: 640x640 1 well, 11.0ms
Speed: 4.2ms preprocess, 11.0ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

imag